In [2]:
import pandas as pd
import numpy as np

## Loading and Pre-processing the data

In [3]:
df = pd.read_excel("C:\\Users\\sheth\\Desktop\\FinBot\\finbot\\docs\\Statement.xlsx")

In [4]:
df

,Transaction Date,Process Date,Transaction Details,Amount £,Transaction Type
0,2024-06-23,2024-06-25,Electric Bill Payment,1263.78,Debit
1,2024-12-14,2024-12-14,Insurance Premium,302.59,Debit
2,2024-07-20,2024-07-20,Local Grocery Store,1159.32,Debit
3,2024-05-16,2024-05-19,Starbucks Coffee,416.39,Debit
4,2024-10-11,2024-10-13,Starbucks Coffee,948.74,Debit
...,...,...,...,...,...
5195,2024-05-14,2024-05-17,Local Grocery Store,286.75,Debit
5196,2024-05-19,2024-05-20,Hotel Booking,410.92,Debit
5197,2024-04-16,2024-04-19,Netflix Subscription,1943.48,Debit
5198,2024-03-28,2024-03-31,Uber Ride,433.70,Debit


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Transaction Date     5200 non-null   datetime64[ns]
 1   Process Date         5200 non-null   datetime64[ns]
 2   Transaction Details  5200 non-null   object        
 3   Amount £             5200 non-null   float64       
 4   Transaction Type     5200 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 203.3+ KB


## Step1: Creating Embeddings

In [6]:
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combine text columns for meaningful context
df['text_for_embedding'] = df['Transaction Details'] + ' - ' + df['Transaction Type']

# Create embeddings for the combined text
embeddings = model.encode(df['text_for_embedding'].tolist(), show_progress_bar=True)

# Add embeddings as a new column
df['embeddings'] = embeddings.tolist()

# Remove the temporary text column
df.drop('text_for_embedding', axis=1, inplace=True)

# Display the shape of embeddings
print(f"Embedding dimensions: {embeddings.shape}")
print(f"\nSample embedding vector (first 5 dimensions):")
print(df['embeddings'].iloc[0][:5])

c:\Users\sheth\Desktop\FinBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 163/163 [00:09<00:00, 17.30it/s]


Embedding dimensions: (5200, 384)

Sample embedding vector (first 5 dimensions):
[-0.030450250953435898, 0.14860986173152924, 0.016605732962489128, -0.055037498474121094, -0.02680222876369953]


In [7]:
df

,Transaction Date,Process Date,Transaction Details,Amount £,Transaction Type,embeddings
0,2024-06-23,2024-06-25,Electric Bill Payment,1263.78,Debit,"[-0.030450250953435898, 0.14860986173152924, 0..."
1,2024-12-14,2024-12-14,Insurance Premium,302.59,Debit,"[-0.05576960742473602, 0.12683182954788208, -0..."
2,2024-07-20,2024-07-20,Local Grocery Store,1159.32,Debit,"[0.01665743999183178, 0.06558531522750854, 0.0..."
3,2024-05-16,2024-05-19,Starbucks Coffee,416.39,Debit,"[-0.026321619749069214, 0.03162455931305885, 0..."
4,2024-10-11,2024-10-13,Starbucks Coffee,948.74,Debit,"[-0.026321619749069214, 0.03162455931305885, 0..."
...,...,...,...,...,...,...
5195,2024-05-14,2024-05-17,Local Grocery Store,286.75,Debit,"[0.01665743999183178, 0.06558531522750854, 0.0..."
5196,2024-05-19,2024-05-20,Hotel Booking,410.92,Debit,"[0.028615882620215416, 0.08039309084415436, -0..."
5197,2024-04-16,2024-04-19,Netflix Subscription,1943.48,Debit,"[-0.03138336166739464, -0.08457714319229126, -..."
5198,2024-03-28,2024-03-31,Uber Ride,433.70,Debit,"[-0.016333548352122307, 0.07169769704341888, 0..."


In [8]:
len(df["embeddings"].iloc[1])

384

In [9]:
df.shape

(5200, 6)

In [10]:
import faiss
import numpy as np

# Convert list of embeddings to numpy array
embedding_array = np.array([emb for emb in df['embeddings'].tolist()]).astype('float32')

# Initialize FAISS index (using L2 distance)
dimension = 384  # dimension from all-MiniLM-L6-v2
index = faiss.IndexFlatL2(dimension)

# Add vectors to the index
index.add(embedding_array)

# Create a search function
def search_similar_transactions(query_text, k=300):
    """
    Search for similar transactions using text query
    Args:
        query_text (str): Text to search for
        k (int): Number of results to return
    """
    # Create embedding for query
    query_embedding = model.encode([query_text]).astype('float32')
    
    # Search in FAISS
    distances, indices = index.search(query_embedding, k)
    
    # Format results
    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        results.append({
            'rank': i+1,
            'distance': float(dist),
            'transaction': df.iloc[idx]['Transaction Details'],
            'amount': df.iloc[idx]['Amount £'],
            'date': df.iloc[idx]['Transaction Date'],
            'type': df.iloc[idx]['Transaction Type']
        })
    
    return results



In [11]:
# Example usage
example_query = "payment"
similar_transactions = search_similar_transactions(example_query)

# Display results
print(f"\nSearch results for: '{example_query}'\n")
for result in similar_transactions:
    print(f"Rank: {result['rank']}")
    print(f"Transaction: {result['transaction']}")
    print(f"Amount: £{result['amount']:.2f}")
    print(f"Date: {result['date']}")
    print(f"Distance: {result['distance']:.2f}\n")

# Save index for later use
faiss.write_index(index, "transaction_index.faiss")



Search results for: 'payment'

Rank: 1
Transaction: Salary Payment
Amount: £828.61
Date: 2024-11-15 00:00:00
Distance: 0.74

Rank: 2
Transaction: Salary Payment
Amount: £1730.19
Date: 2024-09-10 00:00:00
Distance: 0.74

Rank: 3
Transaction: Salary Payment
Amount: £1726.45
Date: 2024-05-22 00:00:00
Distance: 0.74

Rank: 4
Transaction: Salary Payment
Amount: £721.08
Date: 2024-03-24 00:00:00
Distance: 0.74

Rank: 5
Transaction: Salary Payment
Amount: £1834.07
Date: 2025-01-02 00:00:00
Distance: 0.74

Rank: 6
Transaction: Salary Payment
Amount: £1053.05
Date: 2024-06-03 00:00:00
Distance: 0.74

Rank: 7
Transaction: Salary Payment
Amount: £596.08
Date: 2024-12-02 00:00:00
Distance: 0.74

Rank: 8
Transaction: Salary Payment
Amount: £111.24
Date: 2024-06-03 00:00:00
Distance: 0.74

Rank: 9
Transaction: Salary Payment
Amount: £668.93
Date: 2024-12-08 00:00:00
Distance: 0.74

Rank: 10
Transaction: Salary Payment
Amount: £1436.45
Date: 2024-01-12 00:00:00
Distance: 0.74

Rank: 11
Transaction: 

In [ ]:
# import os
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq
# from langchain.schema import SystemMessage, HumanMessage, AIMessage
# # GROQ_API_KEY= ''
# # Load OpenAI API key
# load_dotenv()

# class FinBot:
#     def __init__(self, df, model, faiss_index):
#         self.df = df
#         self.embedding_model = model
#         self.index = faiss_index
#         self.llm = ChatGroq(
#             temperature=0.7,
#             groq_api_key='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',
#             model_name="llama-3.1-8b-instant"  # or "llama2-70b-4096"
#         )
        
#         # Initialize conversation memory with system context
#         self.conversation_history = [
#             SystemMessage(content="""You are a helpful financial assistant analyzing bank transactions.
#             For each query:
#             1. Use the provided transaction data to give accurate insights
#             2. Consider the dates, amounts, transaction details and types in your analysis
#             3. Provide specific examples from the transactions when relevant
#             4. If asked about spending patterns, analyze the amounts and frequencies
#             5. For comparisons, look at similar transactions and their timing""")
#         ]
    
#     def process_query(self, user_input):
#         """Process user query and provide contextual response"""
#         # Add user message to history
#         self.conversation_history.append(HumanMessage(content=user_input))
        
#         # Get relevant transactions
#         relevant_txns = search_similar_transactions(user_input, k=5)
        
#         # Create rich context for LLM
#         context = f"""
#         Based on the transaction data:
        
#         Relevant transactions for your reference:
#         {[f"{t['transaction']} on {t['date']} for £{t['amount']:.2f} ({t['type']})" for t in relevant_txns]}
        
#         Total transactions in database: {len(self.df)}
#         Date range: {self.df['Transaction Date'].min()} to {self.df['Transaction Date'].max()}
        
#         Please provide a detailed analysis based on these transactions.
#         """
        
#         # Get LLM response
#         response = self.llm.generate([self.conversation_history + [
#             SystemMessage(content=context)
#         ]])
        
#         # Add AI response to history
#         self.conversation_history.append(AIMessage(content=response.generations[0][0].text))
        
#         return response.generations[0][0].text

# # Initialize FinBot
# finbot = FinBot(df, model, index)

# # Interactive chat interface
# def chat():
#     print("FinBot: Hi! I can help analyze your transactions. What would you like to know?")
#     print("(Type 'quit' to exit)")
    
#     while True:
#         user_input = input("\nYou: ")
#         if user_input.lower() == 'quit':
#             print("\nFinBot: Goodbye!")
#             break
        
#         try:
#             response = finbot.process_query(user_input)
#             print(f"\nFinBot: {response}")
#         except Exception as e:
#             print(f"\nFinBot: Sorry, I encountered an error: {str(e)}")

# # Start chat
# chat()

FinBot: Hi! I can help analyze your transactions. What would you like to know?
(Type 'quit' to exit)

FinBot: Based on the provided transaction data, I've extracted the relevant information for the given date range (2024-01-01 to 2025-02-04). However, please note that the transaction data provided only includes a limited number of transactions. For a comprehensive analysis, additional data would be required.

Since the relevant transaction data only includes 5 online course transactions, I can provide a summary for these transactions. For the overall database, I will consider the available data.

**Online Course Transactions Summary:**

| Month | Number of Transactions | Total Expenditure (£) |
| --- | --- | --- |
| 2024-01 | 0 | 0 |
| 2024-02 | 0 | 0 |
| 2024-03 | 0 | 0 |
| 2024-04 | 0 | 0 |
| 2024-05 | 1 | £1387.63 |
| 2024-06 | 0 | 0 |
| 2024-07 | 0 | 0 |
| 2024-08 | 0 | 0 |
| 2024-09 | 1 | £313.51 |
| 2024-10 | 0 | 0 |
| 2024-11 | 0 | 0 |
| 2024-12 | 3 | £4031.06 (£1782.05 + £502.7

: 

: 

c:\Users\sheth\Desktop\FinBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-16 14:27:37.256 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 14:27:37.256 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-08-16 14:27:37.256 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 14:27:37.256 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 14

FileNotFoundError: [Errno 2] No such file or directory: 'docs/Statement.xlsx'